<a href="https://colab.research.google.com/github/chodizzle/portfolio1/blob/main/projects/tfid_cats.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install scikit-learn
!pip install unidecode

     |████████████████████████████████| 245kB 24.9MB/s 


In [ ]:
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import json
import unidecode
import time
import re

In [ ]:
%cd drive/MyDrive/

/content/drive/MyDrive


In [ ]:
# df = pd.read_csv('meats_v2.csv', index_col='item_id')

In [ ]:
# df['sw'] = df['sw'].apply(lambda x: x[1:-1].strip().replace('\'', '').split(', '))

In [ ]:
# df.to_pickle('meats_v2.pickle')

In [ ]:
df = pd.read_pickle('meats_v2.pickle')
# pickles are good mmkay

In [ ]:
df.head()

In [ ]:
# df['sw'] = df['sw'].apply(lambda x: ((x.strip('{}').replace('\'', ''))).split(', '))

In [ ]:
# df['sw'] = df['sw'].apply(lambda x: x[2:])

In [ ]:
# df['sw'] = df['sw'].apply(lambda x: [v.lower() for v in x])

In [ ]:
# df.to_csv('meats_v2.csv', index=True)

<h1> TESTING w/ Ground Beef </h1>

In [ ]:
df['category'].value_counts()['Ground Beef']

10063

<h2> IDF & TF Concat </h2>
add custom stop words using category names

combine idf and tf dfs -> analyze for threshold

rinse and repeat on a couple more categories

In [ ]:
test_cats = pd.read_csv('test_cats.csv', index_col=0)
test_cats = test_cats['category'].values
test_cats

In [ ]:
# for x in test_cats:
#   if 'bone' in x.lower(): print(x)

# test_cats[0:5]
# df_test = df[df['category'] == 'Ground Beef']

In [ ]:
df_test = pd.DataFrame()

for cat in test_cats:
  df_temp = df[df['category'] == cat]

  # get stopwords
  sw = df_temp.iloc[0, 2]
  # print(sw)

  # list of all names
  docs = df_temp['name'].values

  #word counts of all names in cat => vectors
  cv = CountVectorizer(token_pattern=r"(?u)\b[a-zA-Z_]+\w+\b|\b\w+[a-zA-Z_]+\b", ngram_range=(1,2), stop_words=sw)  
  # sklearn's default preprocessing converts 't-bone' into 't' and 'bone', which throws up an inconsistency error. I'm OK with it as I want t-bone to be my stop word.
  # print(cv.stop_words) 
  word_count_vectorizer = cv.fit_transform(docs)
  ind = cv.get_feature_names() #indeces for IDF and TF dfs below
  
  #getting IDF only
  tfidf_transformer = TfidfTransformer(smooth_idf=True, use_idf=True)
  tfidf_transformer.fit(word_count_vectorizer)
  df_idf = pd.DataFrame(tfidf_transformer.idf_, index=ind, columns=['idf_weights'])

  # getting TF only
  counts = word_count_vectorizer.toarray().sum(axis=0)
  df_tf = pd.DataFrame(counts, index=ind, columns=['counts'])

  df_com = pd.concat([df_idf, df_tf], axis=1).reset_index()
  df_com['category'] = cat
  df_com['%_cat'] = df_com['counts'] / df_temp.shape[0]
  df_com.rename({'index':'keyword'}, axis=1, inplace=True)

  df_test = df_test.append(df_com)

/usr/local/lib/python3.7/dist-packages/sklearn/feature_extraction/text.py:385: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['bone'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


In [ ]:
df_test.groupby('category').describe()['%_of_cat'].min()

count    586.000000
mean       0.000760
std        0.002267
min        0.000099
25%        0.000099
50%        0.000155
75%        0.000397
max        0.050436
dtype: float64

In [ ]:
df_test.groupby('category').describe()['%_cat']

In [ ]:
df_test.groupby(['category'])['%_cat'].nlargest(2)

category                
Baby Back Ribs      1830    0.420064
                    1302    0.123391
Beef Strip Steak    8090    0.121878
                    8274    0.118558
Bone Marrow         347     0.138000
                              ...   
Tri Tip             1234    0.064541
Wagyu Beef Rib Eye  1162    0.120000
                    163     0.109091
Wild Boar           7863    0.157949
                    2842    0.096582
Name: %_cat, Length: 70, dtype: float64

In [ ]:
df_test['%_cat_med'] = df_test['category'].map(df_test.groupby('category')['%_cat'].median())
df_test['cat_idf_med'] = df_test['category'].map(df_test.groupby('category')['idf_weights'].median())

In [ ]:
df_test.groupby('category').describe()

In [ ]:
df_test.groupby('category').describe()['counts'].min()

count    1451.000000
mean        1.795281
std         3.339908
min         1.000000
25%         1.000000
50%         1.000000
75%         2.000000
max        49.000000
dtype: float64

In [ ]:
df_test.groupby('category').head(25)

,keyword,idf_weights,counts,category,%_cat,%_cat_med,cat_idf_med
1830,rib,1.888530,783,Baby Back Ribs,0.420064,0.000536,7.837869
1302,loin,3.088599,230,Baby Back Ribs,0.123391,0.000536,7.837869
1246,lb,3.252902,196,Baby Back Ribs,0.105150,0.000536,7.837869
306,bf,3.443420,162,Baby Back Ribs,0.086910,0.000536,7.837869
1417,meatball,3.481160,155,Baby Back Ribs,0.083155,0.000536,7.837869
...,...,...,...,...,...,...,...
4181,lb,5.010704,70,Wild Boar,0.018125,0.000259,8.566052
5265,on,5.084812,67,Wild Boar,0.017349,0.000259,8.566052
3940,jumbo,5.054507,67,Wild Boar,0.017349,0.000259,8.566052
2467,farmed,5.069545,66,Wild Boar,0.017090,0.000259,8.566052


In [ ]:
derp = df_test[(df_test['counts'] > 24) & (df_test['idf_weights'] > 6.0)]

In [ ]:
derp.to_csv('derp.csv')

<h2> TFIDF Score </h2>

In [ ]:
count_vector = cv.transform(docs)
# creates a sparse matrix of word counts in documents.... whatever that means XD

tf_idf_vector = tfidf_transformer.transform(count_vector)
# this computes the tf-idf score of docs

In [ ]:
features = cv.get_feature_names()

first_doc_vectors = tf_idf_vector[1]

df=pd.DataFrame(first_doc_vectors.T.todense(), index=features, columns=['tfidf'])

print(df_test.iloc[1, 0], '\n', df[df['tfidf']>0], '\nsum:', df[df['tfidf']>0].values.sum())
# this is the tfidf of first name in df_test

<h2> TFIDF Using Tfidfvectorizer </h2>

In [ ]:
df_temp

In [ ]:
docs1 = df_temp['name'].values
sw1 = df_temp.iloc[0, 2]

In [ ]:
tfidf_vectorizer = TfidfVectorizer(use_idf=True, stop_words=sw1)
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(docs)

In [ ]:
df_temp.shape

(725, 4)

In [ ]:
len(tfidf_vectorizer.get_feature_names())

702

In [ ]:
print(tfidf_vectorizer_vectors.shape)

(725, 702)


In [ ]:
len(tfidf_vectorizer.idf_), len(tfidf_vectorizer.vocabulary_)

(702, 702)

In [ ]:
first_vector_tfidf_vectorizer = tfidf_vectorizer_vectors[0]
df_tfidfv = pd.DataFrame(first_vector_tfidf_vectorizer.T.todense(), index=tfidf_vectorizer.get_feature_names(), columns=['tfidf'])
df_tfidfv[df_tfidfv['tfidf'] > 0]

In [ ]:
# create docs of each cat as collection of all item names in each cat and put into DF
df_docs = pd.DataFrame(columns=['category', 'sw', 'doc'])
df_temp = df[df['category'] == 'Ground Beef']
doc = ' '.join(df_temp['name'].values)
doc = [{'category':'Ground Beef',  'doc': doc}]
df_docs.append(doc, ignore_index=True)

,category,sw,doc
0,Ground Beef,NaN,cab beef grnd chck fine 80/20 f ground whit op...


In [ ]:
# do the same as above but for every cat in test_cats
df_docs = pd.DataFrame(columns=['category', 'sw', 'doc'])

for cat in test_cats: 
  df_temp = df[df['category'] == cat]
  doc = ' '.join(df_temp['name'].values)
  sw = df_temp.iloc[0,2]
  doc_row = [{'category': cat, 'sw' : sw, 'doc': doc}]
  df_docs = df_docs.append(doc_row, ignore_index=True)

df_docs.set_index('category', inplace=True)

TFIDF Vectorizer on df_docs!!!


In [ ]:
# TFIDF given NO stopwords and default preprocess/tokenizer

tfidf_vectorizer = TfidfVectorizer(use_idf=True, token_pattern=r"(?u)\b[a-zA-Z_]+\w+\b|\b\w+[a-zA-Z_]+\b", ngram_range=(1, 2))
tfidf_vectorizer_vectors = tfidf_vectorizer.fit_transform(df_docs['doc'])

In [ ]:
tfidf_vectorizer_vectors

<35x196296 sparse matrix of type '<class 'numpy.float64'>'
	with 262578 stored elements in Compressed Sparse Row format>

In [ ]:
feature_names = tfidf_vectorizer.get_feature_names()

df1 = pd.DataFrame(tfidf_vectorizer_vectors.T.todense(), index=feature_names, columns=test_cats)
df1['total'] = df1.sum(axis=1)
df1.sort_values(by='total', ascending=False)

,Ground Beef,Top Round,Hot Dog,Beef Strip Steak,New York Strip,Flat Iron Steak,Skirt Steak,Flap Meat,Ground Chuck,Wild Boar,Top Sirloin,Short Ribs,Hanger Steak,Chuck Roast,Filet Mignon,Tri Tip,Pork Rack,Chuck Roll,Prime Rib,Baby Back Ribs,Pork Head,Bone Marrow,T-Bone Steaks,Chuck Flat,Spare Ribs,Hot Link,Ham Hocks,Pork Skin,Pork Chops,Pork Neck Bones,Wagyu Beef Rib Eye,Ground Kobe Beef,Lamb,Cornish Game Hens,Pork Chop Center Cut,total
beef,0.659779,0.219269,0.023023,0.597470,0.101513,0.398149,0.453908,0.313509,0.668517,0.000374,0.376033,0.470548,0.415892,0.497382,0.418746,0.291401,0.005485,0.363119,0.374311,0.011746,0.000239,0.472862,0.113386,0.073197,0.012770,0.000452,0.001079,0.009259,0.006195,0.030120,0.237260,0.622446,0.005910,0.000000,0.023745,8.269092
pork,0.013003,0.001161,0.006606,0.005130,0.002125,0.007283,0.015063,0.012280,0.013121,0.000192,0.010401,0.011002,0.002498,0.016344,0.016068,0.002553,0.269827,0.010970,0.033128,0.412005,0.009325,0.028429,0.003835,0.000000,0.438901,0.000465,0.062668,0.346308,0.530297,0.306372,0.000000,0.013208,0.003646,0.000708,0.587066,3.191987
rib,0.007359,0.003923,0.000000,0.043894,0.006008,0.016300,0.020651,0.019514,0.016320,0.001383,0.010695,0.303550,0.014557,0.057494,0.034729,0.006562,0.095456,0.025378,0.445825,0.277814,0.000252,0.045936,0.014984,0.004073,0.230790,0.000000,0.000570,0.003671,0.084329,0.011373,0.198797,0.024693,0.012497,0.000000,0.110687,2.150064
steak,0.005218,0.008016,0.000475,0.112135,0.098386,0.262472,0.309911,0.004816,0.010713,0.000663,0.037267,0.048519,0.323342,0.005938,0.057720,0.010279,0.001996,0.007211,0.167020,0.000000,0.000000,0.035376,0.009706,0.002848,0.000000,0.000000,0.008294,0.001369,0.008244,0.000000,0.027949,0.009670,0.000000,0.000000,0.002553,1.578108
chuck,0.057461,0.006264,0.000000,0.014059,0.010675,0.055318,0.026298,0.058471,0.267570,0.000234,0.007602,0.085829,0.019272,0.236281,0.005592,0.009328,0.000528,0.386531,0.024329,0.000000,0.000000,0.034628,0.001868,0.086244,0.003046,0.000000,0.000000,0.000000,0.000000,0.039069,0.045620,0.014626,0.010363,0.000000,0.000000,1.507108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
install freight,0.000371,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000371
5ir buffalo,0.000371,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000371
5ir bf,0.000371,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000371
blend ab,0.000371,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000371


Testing some preprocessing

In [ ]:
!pip install fuzzywuzzy
!pip install python-Levenshtein
from fuzzywuzzy import fuzz
from fuzzywuzzy import process

     |████████████████████████████████| 51kB 5.4MB/s 
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.2-cp37-cp37m-linux_x86_64.whl size=149825 sha256=c4ee6bffbf1a5fdcd3002e7b26c7fa7b8003b7c47a2c9bea9f1109ecdfc8179e
  Stored in directory: /root/.cache/pip/wheels/b3/26/73/4b48503bac73f01cf18e52cd250947049a7f339e940c5df8fc
Successfully built python-Levenshtein


In [ ]:
df_gb = df_docs.loc['Ground Beef']

df_gb['doc'] = my_preprocess(df_gb['doc'])

sw = df_gb['sw']
to_match = set(df_gb['doc'].split())

In [ ]:
sw

In [ ]:
# sw generator

new_sw = []
# new_sw.append(sw)
for word in sw:
  word = re.sub('[aeiou]', '', word)
  new_sw.append(word)

new_sw

KeyboardInterrupt: ignored

In [ ]:
# preprocess df_gb['doc']

def my_preprocess(text):
  text = re.sub('[^a-zA-Z0-9 %#\.\/\:-]', '', text)
  return text

In [ ]:
scores = {}
for s in sw:
  scores[s] = {}
  for item in to_match:
    score = fuzz.ratio(s, item)
    if score < 75: continue
    scores[s][item] = score

scores 

{'beef': {'%beef': 89,
  '-beef': 89,
  'bbeef': 89,
  'bbef': 75,
  'bee': 86,
  'beed': 75,
  'beedf': 89,
  'beeef': 89,
  'beef': 100,
  'beef-': 89,
  'beef.': 89,
  'beef/': 89,
  'beeg': 75,
  'beep': 75,
  'beer': 75,
  'beet': 75,
  'bef': 86,
  'beff': 75,
  'breef': 89,
  'eef': 86,
  'feef': 75,
  'g.beef': 80,
  'grbeef': 80,
  'hpbeef': 80,
  'reef': 75,
  'veef': 75},
 'ground': {'6round': 83,
  'background': 75,
  'fbfground': 80,
  'fround': 83,
  'gound': 91,
  'graound': 92,
  'grnd': 80,
  'grojnd': 83,
  'groun': 91,
  'ground': 100,
  'ground-1lb': 75,
  'ground-abf': 75,
  'ground10lb': 75,
  'grounds': 92,
  'groupd': 83,
  'grpound': 92,
  'round': 91,
  'ups-ground': 75}}

In [ ]:
process.extract?